<a href="https://colab.research.google.com/github/Soumadip-das-003/TravelPlanner/blob/main/TravelPranner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
pip install -q streamlit google-generativeai pyngrok


In [56]:
import os

os.environ["PROVIDER"] = "gemini"

# API keys (set at least one)
os.environ["GEMINI_API_KEY"] = "GEMINI_KEY"


In [57]:
from pyngrok import ngrok

!ngrok authtoken YOUR_NGROK_AUTH_TOKEN

public_url = ngrok.connect(8501)
print("🌍 Public URL →", public_url.public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌍 Public URL → https://462c766068b4.ngrok-free.app


In [58]:
%%writefile config.py
import os

class Config:
    PROVIDER = os.environ.get("PROVIDER", "gemini").lower()

    # Gemini
    GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY", "")
    GEMINI_MODEL = "gemini-pro"

    DEFAULT_CURRENCY = "INR"
    MAX_DAYS = 14
    MIN_DAYS = 1

Overwriting config.py


In [59]:
%%writefile ai_client.py
from config import Config
import google.generativeai as genai
from openai import OpenAI
import json
import re

class AIClient:
    def __init__(self):
        self.provider = Config.PROVIDER
        if self.provider == "gemini":
            if not Config.GEMINI_API_KEY:
                raise ValueError("Gemini API key not set.")
            genai.configure(api_key=Config.GEMINI_API_KEY)
            self.model = genai.GenerativeModel(Config.GEMINI_MODEL)


        else:
            raise ValueError("Unsupported provider.")

    def generate_itinerary(self, prompt):
        if self.provider == "gemini":
            resp = self.model.generate_content(prompt)
            text = getattr(resp, "text", "") or ""
            return self._extract_outputs(text)


    def _extract_outputs(self, text):
        json_match = re.search(r"```json\s*(\{.*?\})\s*```", text, flags=re.DOTALL)
        itinerary_json = {}
        if json_match:
            try:
                itinerary_json = json.loads(json_match.group(1))
            except:
                itinerary_json = {"error": "Failed to parse JSON"}

        summary_text = text[json_match.end(0):].strip() if json_match else text.strip()
        return {"itinerary_json": itinerary_json, "summary_text": summary_text}

Overwriting ai_client.py


In [60]:
%%writefile planner.py
from config import Config

def validate_inputs(data):
    destination = (data.get("destination") or "").strip()
    start_date = (data.get("start_date") or "").strip()
    duration_days = int(data.get("duration_days") or 0)
    budget_level = (data.get("budget_level") or "tight").strip().lower()
    interests = [i.strip().lower() for i in (data.get("interests") or "").split(",") if i.strip()]
    transport = (data.get("transport") or "bus/train").strip().lower()
    stay_type = (data.get("stay_type") or "hostel").strip().lower()
    currency = Config.DEFAULT_CURRENCY

    if not destination:
        return False, "Destination is required.", None
    if duration_days < Config.MIN_DAYS or duration_days > Config.MAX_DAYS:
        return False, f"Duration must be between {Config.MIN_DAYS} and {Config.MAX_DAYS} days.", None

    return True, "", {
        "destination": destination,
        "start_date": start_date,
        "duration_days": duration_days,
        "budget_level": budget_level,
        "interests": interests,
        "transport": transport,
        "stay_type": stay_type,
        "currency": currency
    }

def build_prompt(params):
    destination = params["destination"]
    duration = params["duration_days"]
    budget = params["budget_level"]
    interests = ", ".join(params["interests"]) if params["interests"] else "general sightseeing"
    transport = params["transport"]
    stay = params["stay_type"]
    currency = params["currency"]
    start_date = params["start_date"] or "an upcoming weekend"

    return f"""
You are a student-focused travel planner. Create a budget-friendly {duration}-day itinerary for {destination} starting on {start_date}.

Constraints:
- Budget level: {budget}.
- Interests: {interests}.
- Preferred transport: {transport}.
- Preferred stay: {stay}.
- Currency: {currency}.
- Prioritize free/low-cost highlights, student discounts, and safety.

Output format:
1) JSON fenced block with daily plan, meals, transport, accommodation, estimated costs, and tips.
2) Below JSON, a concise student-friendly summary (5-7 sentences).
"""

Overwriting planner.py


In [61]:
%%writefile app.py
import streamlit as st
from ai_client import AIClient
from planner import validate_inputs, build_prompt

st.set_page_config(page_title="Student AI Travel Planner", layout="wide")
st.title("🎓 Student AI Travel Planner")

with st.form("planner_form"):
    destination = st.text_input("Destination", placeholder="e.g., Ooty, Goa, Jaipur")
    start_date = st.date_input("Start date (optional)", value=None)
    duration_days = st.number_input("Duration (days)", min_value=1, max_value=14, value=3)
    budget_level = st.selectbox("Budget level", ["tight", "moderate", "flexible"])
    transport = st.selectbox("Preferred transport", ["bus/train", "shared cab", "flight"])
    stay_type = st.selectbox("Preferred stay", ["hostel", "homestay", "budget hotel"])
    interests = st.text_input("Interests (comma-separated)", placeholder="nature, food, culture, history")
    submitted = st.form_submit_button("Generate Itinerary")

if submitted:
    ok, msg, normalized = validate_inputs({
        "destination": destination,
        "start_date": str(start_date) if start_date else "",
        "duration_days": int(duration_days),
        "budget_level": budget_level,
        "interests": interests,
        "transport": transport,
        "stay_type": stay_type
    })

    if not ok:
        st.error(msg)
    else:
        prompt = build_prompt(normalized)
        try:
            client = AIClient()
            result = client.generate_itinerary(prompt)
            st.subheader("📋 JSON Itinerary")
            st.json(result["itinerary_json"])
            st.subheader("📝 Student Summary")
            st.write(result["summary_text"])
        except Exception as e:
            st.error(f"Error: {e}")

Overwriting app.py


In [ ]:
!streamlit run app.py --server.port 8501 --server.headless true




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.201.233:8501

